# QPROP: Propeller Analysis Code

Mark Deela wrote the *QProp* application to help design propeller/electric motor combinations for powered flight. The last release of this code was produced in 2009 (*Qprop 1.31*). In this note we will look at the part of this code that deals with the propeller design. The intent here is to work toward code that can be used for indoor propeller design.

We will start this work by making sure the original code can be compiled and run on my Macbook. Later, we will test this process on Windows as well.

## Download Qprop v1.22

In [11]:
!curl https://web.mit.edu/drela/Public/web/qprop/qprop1.22.tar.gz -o ../master/qprop1.22.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  211k  100  211k    0     0  1227k      0 --:--:-- --:--:-- --:--:-- 1303k


## Extract the Code

In [12]:
! cd ../master && tar zxvf qprop1.22.tar.gz

x Qprop/
x Qprop/README
x Qprop/bin/
x Qprop/bin/Makefile
x Qprop/qmil_doc.txt
x Qprop/qprop_doc.txt
x Qprop/runs/
x Qprop/runs/air/
x Qprop/runs/air/apc80.dat
x Qprop/runs/air/apc1.dat
x Qprop/runs/air/apc2.dat
x Qprop/runs/air/gunther.dat
x Qprop/runs/air/propi.dat
x Qprop/runs/air/s9.dat
x Qprop/runs/kagan_cruise.txt
x Qprop/runs/airship.mil
x Qprop/runs/apc.txt
x Qprop/runs/apc14x10e
x Qprop/runs/apc16x10e
x Qprop/runs/apc18x10e
x Qprop/runs/apc5.5x4.5
x Qprop/runs/apc6x4
x Qprop/runs/apc7x4
x Qprop/runs/apc8x3.8
x Qprop/runs/apc8x6
x Qprop/runs/apc8x8
x Qprop/runs/axi2204-54
x Qprop/runs/axi4130-16
x Qprop/runs/axi4130-20
x Qprop/runs/axi5320-28
x Qprop/runs/cam6x3
x Qprop/runs/cam6x3mod
x Qprop/runs/cam6x4
x Qprop/runs/cam6x7
x Qprop/runs/df1.mil
x Qprop/runs/dfpar1
x Qprop/runs/dfpar2
x Qprop/runs/dfprop
x Qprop/runs/ep1280
x Qprop/runs/f1d
x Qprop/runs/f1d.mil
x Qprop/runs/f1dVD0
x Qprop/runs/f1dVD1
x Qprop/runs/fid
x Qprop/runs/gunther
x Qprop/runs/kagan
x Qprop/runs/kagan.raw

In [14]:
! cd ../master/Qprop/bin && cat Makefile


SRC = ../src
VLSRC = ../vlsrc
FFLAGS = -O -r8 

#------------------------------------
# default Unix fortran
FC = f77
FTNLIB =

#------------------------------------
# Intel Fortran Compiler
FC = ifort
#FTNLIB = -Vaxlib
#FTNLIB = -Vaxlib /usr/lib/C-ctype.o /usr/lib/C_name.o /usr/lib/ctype-info.o

#------------------------------------
#FC = g77
#FFLAGS = -O -dbl

#------------------------------------

qprop: qprop.o gvcalc.o cdfun.o tqcalc.o motor.o spline.o io.o qcget.o
	$(FC) -o qprop qprop.o gvcalc.o cdfun.o tqcalc.o motor.o \
spline.o io.o qcget.o $(FTNLIB)

qmil: qmil.o tpdes.o gvcalc.o cdfun.o tqcalc.o bnsolv.o \
spline.o io.o qcget.o
	$(FC) -o qmil qmil.o tpdes.o gvcalc.o cdfun.o tqcalc.o bnsolv.o \
spline.o io.o qcget.o $(FTNLIB)

vlprop: vlprop.o cdfun.o vlset.o motor.o aic.o \
spline.o io.o qcget.o aread.o lefinda.o
	$(FC) -o vlprop vlprop.o cdfun.o vlset.o motor.o aic.o \
spline.o io.o qcget.o aread.o lefinda.o $(FTNLIB)



qprop.o: $(SRC)/qprop.f $(SRC)/QDEF.INC
	$(FC) -c $

Looking at this file, we see that several programs are constructed using this code: **qprop**, **qmil**, and **vlprop**. However, the code needed for that last program does not seem to be present in the archive. 

*Qmil* is a design program that generates data files to be processed by *Qprop*. For the moment, we will focus only on the *Qprop* code.

## Compiling Qprop

The **Makefile found in the **bin** directory is set up to use the Intel Fortran compiler. We will use *GFortran* which is part of the *Gnu* tool chain.

Here is a check that **gfortran** is available:

In [27]:
! gfortran -v

Using built-in specs.
COLLECT_GCC=gfortran
COLLECT_LTO_WRAPPER=/usr/local/Cellar/gcc/12.2.0/bin/../libexec/gcc/x86_64-apple-darwin21/12/lto-wrapper
Target: x86_64-apple-darwin21
Configured with: ../configure --prefix=/usr/local/opt/gcc --libdir=/usr/local/opt/gcc/lib/gcc/current --disable-nls --enable-checking=release --with-gcc-major-version-only --enable-languages=c,c++,objc,obj-c++,fortran --program-suffix=-12 --with-gmp=/usr/local/opt/gmp --with-mpfr=/usr/local/opt/mpfr --with-mpc=/usr/local/opt/libmpc --with-isl=/usr/local/opt/isl --with-zstd=/usr/local/opt/zstd --with-pkgversion='Homebrew GCC 12.2.0' --with-bugurl=https://github.com/Homebrew/homebrew-core/issues --with-system-zlib --build=x86_64-apple-darwin21 --with-sysroot=/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk
Thread model: posix
Supported LTO compression algorithms: zlib zstd
gcc version 12.2.0 (Homebrew GCC 12.2.0) 


Building *Qprop* is fairly easy. In real development, we would use a tool like *GNU Make* to set up the build. However, we will let Python do things directly in this page to show what steps are needed.

### Copy Source Files

In order to leave the master files untouched, we will copy the needed source files into a build directory:

In [44]:
!mkdir -p ../master/build-qprop

From the master **Makefile**, here are the files needed to build the program:

In [45]:
FSRCS = [
    "qprop.f", 
    "gvcalc.f", 
    "cdfun.f", 
    "tqcalc.f", 
    "motor.f",
    "spline.f", 
    "io.f", 
    "qcget.f"
]

Copy the files to the build directory

In [46]:
import shutil
BUILD = "../master/build-qprop/"
for f in FSRCS:
    print(f"Copying {f}")
    src = "../master/Qprop/src/" + f
    dst = BUILD + f
    shutil.copyfile(src, dst)

Copying qprop.f
Copying gvcalc.f
Copying cdfun.f
Copying tqcalc.f
Copying motor.f
Copying spline.f
Copying io.f
Copying qcget.f


### Generate Object Files

Now, we generate the object files. This first step deletes any old object files that may be present:

In [50]:
import os
cmd = f"rm -f {BUILD}*.o"
cmd
ret = os.system(cmd)

In [51]:
for f in FSRCS:
    src = f"{BUILD}{f}"
    dst = f"{BUILD}{f}.o"
    cmd = f"gfortran -c {src} -o {dst}"
    ret = os.system(cmd)

### Link to Build Executable

Finally, we link these object files together to build the application:

In [52]:
objs = ""
for f in FSRCS:
    objs += f"{BUILD}{f}.o "
cmd = f"gfortran {objs} -o {BUILD}qprop"
ret = os.system(cmd)

This creates the program executable. 

### Test the Application

We can test the program using an example input file from the master copy:

In [61]:
!../master/build-qprop/qprop ../master/Qprop/runs/cam6x3


 Motor file not found:                                                  
 Default motor used  :  Speed-400 3321 (6V) direct drive                                                

 Run parameter file not found:                                                 
 Default velocities, voltages, pitch used

# QPROP Version 1.22
# 
# Graupner CAM 6x3 folder                                                         
# 
# Speed-400 3321 (6V) direct drive                                                
#   0.31000     R  (Ohm)                        
#   0.77000     Io (Amp)                        
#    2760.0     Kv (rpm/Volt)                   
# 
#   rho =  1.2250     kg/m^3
#   mu  = 0.17800E-04 kg/m-s
#   a   =  340.00     m/s   
# 
#  1         2        3          4          5         6            7         8       9        10        11        12          13        14        15      16          17           18      19
# 
#  V(m/s)    rpm      Dbeta      T(N)       Q(N-m)    Pshaft(W)    Volt